In [331]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from mne.stats import permutation_cluster_test
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction
import normalization
import anal_functions

### Add Directories/ Load Files

In [332]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


### Select Peaks and Plot Analytic Signal

In [ ]:
#Load raw fif data

test_raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'without_med_FTG',
        'Sub050_ARTREJECT_FIF.fif'
    )
)

subID = 'Sub050'
fft_name = str(subID) + '_'
print(fft_name)

In [ ]:
reload(dat_preproc)
x = test_raw.get_data() 
x1 = x[0,:]

#peakMed = 80
peakStim = 63

#dat_ngam = dat_preproc.low_highpass_filter(x1, peakMed-2, peakMed+2) 
dat_subh = dat_preproc.low_highpass_filter(x1, peakStim-2, peakStim+2) 
#dat_inb = dat_preproc.low_highpass_filter(x1, peakStim+3, peakMed-3) 

#datall = [dat_ngam, dat_subh, dat_inb] 
#labels = ['Peak'+str(peakMed)+'Hz','Peak'+str(peakStim)+'Hz', str(peakStim+3) + '-' + str(peakMed-3)+'Hz']

datall = [dat_subh] 
labels = ['Peak'+str(peakStim)+'Hz']

print(labels)

In [ ]:
def window_rms(a, window_size):
  a2 = np.power(a,2)
  window = np.ones(window_size)/float(window_size)
  return np.sqrt(np.convolve(a2, window, 'valid'))

In [ ]:
%matplotlib qt
sm_signal_np = np.empty(shape = (1, x1.shape[0] - 499))
sm_signal_np[:] = np.nan

fig, axes = plt.subplots(1, 1, figsize=(12, 5))
wintosmooth = 500

for idx, dat in enumerate(datall):
    hiltr = hilbert(dat)
    amplitude_envelope = np.abs(hiltr)
    zscore_sign = stats.zscore(np.squeeze(amplitude_envelope))

    sm_signal = window_rms(zscore_sign, wintosmooth)
    
    plt.plot(sm_signal, label = labels[idx])
    #plt.plot(np.arange(0,75000), amplitude_envelope, label = labels[idx]) 
    
    #axes[idx].axvline(26250, color = 'b', ls='--', lw=2, label = 'Stim On')
    #axes[idx].axvline(50250, color = 'g', ls='--', lw=2, label = 'Stim Off')
    plt.ylabel('Analytic Signal')
    plt.xlim([0, sm_signal.shape[0]])

    
    sm_signal_np[idx,:] = sm_signal

    #axes[idx].set_xticks(ticks = np.arange(0, 80000, 10000), labels = np.arange(0,320,40))
    plt.xlabel('Time [sec]')

    

plt.suptitle('Smoothing Window: 500 samples')
plt.legend(loc='upper right')

plt.show()


In [ ]:
x2 = x[4, :] 
sm_stim = window_rms(x2, wintosmooth)
sm_stim1 = (sm_stim)/3

In [ ]:
fig, ax1 = plt.subplots(figsize = (18,6))
#plt.rcParams['font.size'] = 10
ax2 = ax1.twinx()
for idx, dat in enumerate(sm_signal_np):
    ax1.plot(sm_signal_np[idx,:], label = labels[idx], lw = 2)
ax2.plot(sm_stim1[:], label = 'Stimulation', color = 'grey', ls='--', lw=3, alpha = 0.4)
ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
#ax2.set_yticks(np.arange(0.5, 2.5, 0.25))
#ax2.set_yticklabels(np.arange(0.25, 2.25, 0.25))
#ax1.set_xlim(0, sm_signal_np.shape[0])
#ax1.set_xticks(np.arange(0, 100000, 20000))
#ax1.set_xticklabels(np.arange(0, 400, 80))
ax1.set_xlabel('Time [samples]')
plt.title(str(subID))
plt.show()

In [ ]:
#sm_analSignal = np.transpose(np.squeeze(np.array([[sm_signal_np[0]], [sm_signal_np[1]],[sm_signal_np[2]],[sm_stim1]])))
sm_analSignal = np.transpose(np.squeeze(np.array([[sm_signal_np[0]],[sm_stim1]])))
sm_analSignal_df = pd.DataFrame(sm_analSignal, 
    columns = ['StimOn','StimVec'],
    )
print(sm_analSignal_df)

In [ ]:
fft_fig = os.path.join(project_path, 'figures','anal_signal','without_med_FTG/')
fft_file = os.path.join(project_path, 'data','anal_signal','without_med_FTG/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignal',dpi = 300)
sm_analSignal_df.to_csv(str(fft_file)+str(fft_name)+'sm_analSignal.csv')

### Crop Analytic Signal to Epochs of Interest

In [ ]:
##Convert analytic signal arrays to mne objects:
anal_signal_df = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'without_med_FTG',
    'Sub050_sm_analSignal.csv'
))

df_filtered = anal_signal_df.iloc[:, 1:]
dat_anal = df_filtered.values
ch_names = list(df_filtered.columns)
sfreq = 250
info = mne.create_info(ch_names, sfreq)
raw_anal = mne.io.RawArray(dat_anal.T, info)


In [ ]:
fif_name = os.path.join(project_path, 'data','anal_signal','without_med_FTG','Sub050'+'_AnalFIF.fif')
print(fif_name)

In [ ]:
raw_anal.save(fif_name)

In [ ]:
anal_epochs = pd.read_excel(os.path.join(
    project_path,
    'data', 'anal_signal',
    'Anal_epochs.xlsx'
))

anal_epochs
#subID = 'Sub005'
#fft_name = str(subID) + '_'
#print(fft_name)

In [341]:
'''anal_file = os.path.join(project_path, 'data','anal_signal', 'with_med_FTG/')
anal_file = os.path.join(project_path, 'data','anal_signal', 'with_med_FTG/')'''

anal_fif = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'anal_signal',
        'with_med_FTG',
        'Sub065_AnalFIF.fif'
    )
)

subID = 'Sub007'
%matplotlib qt
anal_fif.plot(duration = 200)
'''cropped_anal_1tp = anal_functions.anal_transitions_1tp(anal_epochs, anal_fif, subID, 20) 

np.save(str(anal_file) + str(subID)+'sm_analSignal1TP.npy', cropped_anal_1tp)
plt.savefig(str(anal_fig) + str(subID)+'sm_analSignal2TP',dpi = 150)'''

Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub065_AnalFIF.fif...
Isotrak not found
    Range : 0 ... 113200 =      0.000 ...   452.800 secs
Ready.


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_15264\3838563601.py:4: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub065_AnalFIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  anal_fif = mne.io.read_raw_fif(os.path.join(


"cropped_anal_1tp = anal_functions.anal_transitions_1tp(anal_epochs, anal_fif, subID, 20) \n\nnp.save(str(anal_file) + str(subID)+'sm_analSignal1TP.npy', cropped_anal_1tp)\nplt.savefig(str(anal_fig) + str(subID)+'sm_analSignal2TP',dpi = 150)"

Channels marked as bad:
none


In [ ]:
import glob
anal_file = os.path.join(project_path, 'data','anal_signal', 'SWITCH_OFF/')
anal_fig = os.path.join(project_path, 'figures','anal_signal', 'with_med_FTG/')


directory = os.path.join(project_path,
                         'data',
                         'anal_signal',
                         'with_med_FTG')  # Update with your directory path

# Create a file pattern to match .fif files
file_pattern = '*.fif'

# Get a list of file paths that match the pattern
file_list = glob.glob(os.path.join(directory, file_pattern))

# Loop through the file list
for file_path in file_list:
    
    file_name = os.path.basename(file_path)
    subID = file_name[:6]

    anal_fif = mne.io.read_raw_fif(file_path)

    cropped_anal_1tp = anal_functions.anal_transitions_1tp(anal_epochs, anal_fif, subID, 10)

    np.save(str(anal_file) + str(subID)+'sm_analSignalSWITCH_OFF.npy', cropped_anal_1tp)
    plt.savefig(str(anal_fig) + str(subID)+'sm_analSignalSWITCH_OFF',dpi = 150)

    

In [ ]:
reload(anal_functions)
cropped_anal_2tp = anal_functions.anal_transitions_2tp(anal_epochs, anal_fif, subID, 20)


In [ ]:
anal_fig = os.path.join(project_path, 'figures','anal_signal', 'with_med_FTG/')
anal_file = os.path.join(project_path, 'data','anal_signal', 'with_med_FTG/')

plt.savefig(str(anal_fig) + str(subID)+'sm_analSignal2TP',dpi = 150)
np.save(str(anal_file) + str(subID)+'sm_analSignal2TP.npy', cropped_anal_2tp)

In [ ]:
reload(anal_functions)
cropped_anal_1tp = anal_functions.anal_transitions_1tp(anal_epochs, anal_fif, subID, 20)

In [ ]:
#plt.savefig(str(anal_fig) + str(subID)+'sm_analSignal1TP',dpi = 150)
anal_file = os.path.join(project_path, 'data','anal_signal', 'with_med_FTG/')
np.save(str(anal_file) + str(subID)+'sm_analSignal1TP.npy', cropped_anal_1tp)

### Average Cropped Analytic Signal and Plot

In [ ]:
# Specify the directory path
directory = os.path.join(project_path, 'data', 'anal_signal', 'switch_off')

# Find all .npy files in the directory ending with 'analSignal2TP.npy'
file_list = [file for file in os.listdir(directory) if file.endswith('SWITCH_OFF.npy')]

# Create an empty list to store the data arrays
data_list = []

# Loop through the file list and load the data
for file in file_list:
    data = np.load(os.path.join(directory, file))

    if data.shape[0] == 4:
        my_vec = 1
    else:
        my_vec == 0

    data_list.append(data[my_vec,:])  # Select the first 10000 columns

# Stack the data arrays along the third axis
stacked_data = np.stack(data_list, axis=0)

# Calculate the mean and standard error over the 3rd dimension
mean_data = np.mean(stacked_data, axis=0)
sem_data = np.std(stacked_data, axis=0) / np.sqrt(stacked_data.shape[0])

plt.plot(np.arange(1,5002), mean_data, label='Subharmonic')
plt.fill_between(np.arange(1,5002),mean_data - sem_data, mean_data + sem_data, alpha=0.3)

plt.axvline(x=2500, color='grey', linestyle='--', lw = 3, alpha = 0.4, label = 'Stim Off')

plt.xlim(0,5000)
plt.xticks(np.arange(0,5500,500), labels=np.arange(-10,12,2))
plt.xlabel('Time [sec]')
plt.ylabel('Z-scored Smoothed Analytic Signal')
plt.legend()
plt.show()

In [ ]:
%matplotlib qt

# Plot the data
plt.figure(figsize=(12, 6))
x = np.arange(mean_data.shape[0]) + 1  # Adjusted to start from 1

labels = ['Spontaneous FTG', 'Stim-induced FTG', 'In-Between Activity']

for i in range(mean_data.shape[0]):
    plt.plot(x, mean_data[i], label=labels[i])

for i in range(mean_data.shape[0]):
    plt.fill_between(x, mean_data[i] - sem_data[i], mean_data[i] + sem_data[i], alpha=0.3)

plt.axvline(x=5000, color='grey', linestyle='--', lw = 3, alpha = 0.4, label = 'Subharmonic On')

plt.xlim(0,10000)
plt.xticks(np.arange(0,10500,1000), labels=np.arange(-20,24,4))
plt.xlabel('Time [sec]')
plt.ylabel('Z-scored Smoothed Analytic Signal')
plt.legend()
plt.show()

In [ ]:
fft_fig = os.path.join(project_path, 'figures', 'anal_signal\\')

plt.savefig(str(fft_fig)+'AVG_AnalCropped_SWITCH_OFF',dpi = 300)
plt.savefig(str(fft_fig)+'AVG_AnalCropped_SWITCH_OFF.pdf', format='pdf')

In [ ]:
pval = 0.05  # arbitrary
dfn = 2 - 1  # degrees of freedom numerator
dfd = med_anal.shape[0] - 2  # degrees of freedom denominator
thresh = scipy.stats.f.ppf(1 - pval, dfn=dfn, dfd=dfd)  # F distribution
print(thresh)

In [ ]:
perm_nd = np.empty([6,med_anal.shape[0]])
perm_nd[:] = np.nan
perm_nd[0,:] = np.transpose(med_anal)

In [ ]:
y = np.expand_dims(med_anal, axis = 2)
y1 = np.expand_dims(stim_anal, axis = 2)


In [ ]:
#H0 = np.empty([1000,6])
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([y, y1], n_permutations=1000,
                             threshold=None, n_jobs=None)


In [ ]:
cluster_p_values

### Make all Signals same Length

In [ ]:
import traces

In [342]:
anal_epochs = pd.read_excel(os.path.join(
    project_path,
    'data', 'anal_signal',
    'Anal_epochs.xlsx'
))

anal_epochs

,Percept_ID,Stim_On,Subh_On,preSub_On,Switch_Off
0,Sub005,127.214,279.537,138.311,361.148
1,Sub006,129.129,534.579,407.466,1023.902
2,Sub007,NaN,535.861,419.381,671.748
3,Sub009,124.735,644.723,514.354,667.762
4,Sub014,158.088,739.806,608.564,759.790
5,Sub017,55.325,222.195,147.549,327.998
6,Sub021,NaN,607.155,482.054,859.492
7,Sub025,NaN,468.856,369.419,696.179
8,Sub028,NaN,331.677,121.758,787.140
9,Sub029,32.987,125.749,65.015,281.606


In [377]:
import glob

directory = os.path.join(project_path,
                         'data',
                         'anal_signal',
                         'with_med_FTG//')  # Update with your directory path

# Create a file pattern to match .fif files
file_pattern = '*.fif'

# Get a list of file paths that match the pattern
file_list = glob.glob(os.path.join(directory, file_pattern))

# Loop through the file list
for file_path in file_list:
    
    file_name = os.path.basename(file_path)
    subID = file_name[:6]

    anal_fif = mne.io.read_raw_fif(file_path)

    filtered_df = anal_epochs[anal_epochs['Percept_ID'] == subID]
    
    rec_on = 1
    stim_on = filtered_df['Stim_On'].values[0]
    subh_on = filtered_df['Subh_On'].values[0]
    stim_off = filtered_df['Switch_Off'].values[0]
    rec_off = anal_fif.times[-1]

    part1 = anal_fif.copy().crop(tmin=rec_on, tmax = stim_on).get_data()
    part2 = anal_fif.copy().crop(tmin=stim_on, tmax = subh_on).get_data()
    part3 = anal_fif.copy().crop(tmin=subh_on, tmax = stim_off).get_data()
    part4 = anal_fif.copy().crop(tmin=stim_off, tmax = rec_off).get_data()

''''''    np.save(f'{directory}{subID}-Anal_Part1.npy', part1)
    np.save(f'{directory}{subID}-Anal_Part2.npy', part2)
    np.save(f'{directory}{subID}-Anal_Part3.npy', part3)
    np.save(f'{directory}{subID}-Anal_Part4.npy', part4)'''
'''

    

Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub005_AnalFIF.fif...
Isotrak not found
    Range : 0 ... 174125 =      0.000 ...   696.500 secs
Ready.
Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub006_AnalFIF.fif...
Isotrak not found
    Range : 0 ... 288813 =      0.000 ...  1155.252 secs
Ready.


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_15264\3350545071.py:20: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub005_AnalFIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  anal_fif = mne.io.read_raw_fif(file_path)
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_15264\3350545071.py:20: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub006_AnalFIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  anal_fif = mne.io.read_raw

Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub009_AnalFIF.fif...
Isotrak not found
    Range : 0 ... 198687 =      0.000 ...   794.748 secs
Ready.
Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub014_AnalFIF.fif...
Isotrak not found
    Range : 0 ... 230511 =      0.000 ...   922.044 secs
Ready.
Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub017_AnalFIF.fif...
Isotrak not found
    Range : 0 ... 91577 =      0.000 ...   366.308 secs
Ready.
Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub029_AnalFIF.fif...


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_15264\3350545071.py:20: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub009_AnalFIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  anal_fif = mne.io.read_raw_fif(file_path)
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_15264\3350545071.py:20: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub014_AnalFIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  anal_fif = mne.io.read_raw

Isotrak not found
    Range : 0 ... 73472 =      0.000 ...   293.888 secs
Ready.
Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub045_AnalFIF.fif...
Isotrak not found
    Range : 0 ... 145112 =      0.000 ...   580.448 secs
Ready.
Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub065_AnalFIF.fif...
Isotrak not found
    Range : 0 ... 113200 =      0.000 ...   452.800 secs
Ready.


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_15264\3350545071.py:20: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub029_AnalFIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  anal_fif = mne.io.read_raw_fif(file_path)
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_15264\3350545071.py:20: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\anal_signal\with_med_FTG\Sub045_AnalFIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  anal_fif = mne.io.read_raw

"    np.save(f'{directory}{subID}-Anal_Part1.npy', part1)\n    np.save(f'{directory}{subID}-Anal_Part2.npy', part2)\n    np.save(f'{directory}{subID}-Anal_Part3.npy', part3)\n    np.save(f'{directory}{subID}-Anal_Part4.npy', part4)"

In [381]:
### resample with traces
import traces

# Specify the directory path
directory = os.path.join(
    project_path,
    'data', 'anal_signal', 'with_med_FTG'
)

# Find all files in the directory ending with 'Anal_Part1.npy'
file_list = [file for file in os.listdir(directory) if file.endswith('Anal_Part1.npy')]

# Create an empty list to store the arrays
arrays = []

# Loop through the file list and load the arrays
for file in file_list:
    array = np.load(os.path.join(directory, file))
    arrays.append(array)

# Find the maximum length among all arrays
max_length = max(arr.shape[1] for arr in arrays)

# Create a new list to store the resized arrays
resized_arrays = []

# Resize each array to the maximum length
for arr in arrays:
    resized_arr = np.resize(arr, (4, max_length))
    resized_arrays.append(resized_arr)

# Print the resized arrays
for i, arr in enumerate(resized_arrays):
    print(f"Resized Array {i+1}:", arr)

Resized Array 1: [[1.25052142 1.25218332 1.25378287 1.25532186]
 [1.25680101 1.2582233  1.25959516 1.26091981]
 [1.26219749 1.26343143 1.26462686 1.26578748]
 ...
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]
Resized Array 2: [[1.84965765 1.84990466 1.84984553 1.84948814]
 [1.84885573 1.84795797 1.84679663 1.845384  ]
 [1.84372628 1.84183323 1.83973098 1.83743131]
 ...
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]
Resized Array 3: [[1.67163801 1.67102385 1.67045951 1.66995382]
 [1.66951358 1.66915071 1.66887498 1.66869366]
 [1.66861689 1.66865277 1.66880286 1.6690706 ]
 ...
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]
Resized Array 4: [[0.9150902  0.91125357 0.90743232 0.90362781]
 [0.89984554 0.896092

In [383]:
for i, arr in enumerate(resized_arrays):
    print(arr.shape)
    plt.plot(arr[:,0])

(58252, 4)
(58252, 4)
(58252, 4)
(58252, 4)
(58252, 4)
(58252, 4)
(58252, 4)
(58252, 4)


### Switching Stim Off Analytic Signal for all

In [ ]:
stim_anal = []

ps_path = os.path.join(
   project_path,
   'data', 'anal_signal', 'switch_off'
)

for filename in os.listdir(ps_path):
   if filename.endswith('SWITCH_OFF.csv'):
   #with open(os.path.join(ps_path, filename), 'r') as f: # open in readonly mode
      this_df = pd.read_csv(os.path.join(ps_path,filename))
      this_stim_anal = this_df['StimOn']

      stim_anal.append(this_stim_anal)



In [ ]:
all_stim_anal_df = pd.DataFrame(stim_anal)

mean_values = all_stim_anal_df.mean(skipna=True)
sem_values =  all_stim_anal_df.sem(skipna=True)


In [ ]:

# Plot the mean with shaded area for 
plot_mean = mean_values[:]
plot_sem = sem_values[:]
plt.plot(plot_mean, color='blue', label = 'Stim-Induced FTG')
plt.fill_between(plot_mean.index, plot_mean - plot_sem, plot_mean + plot_sem, color='lightblue')
plt.axvline(x = 5000, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Stim Off')
plt.ylim(0.5,2)
plt.xlim(2500,7500)
plt.xticks(np.arange(2500,8000,500), labels=np.arange(-10,12,2))
plt.legend()
plt.xlabel('Time [sec]')
plt.ylabel('Z-scored Smoothed Analytic Signal')

In [ ]:
fft_fig = os.path.join(project_path, 'results\\')

plt.savefig(str(fft_fig)+'SWITCH_STIM_OFF',dpi = 250)